# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# Загружаем библиотеки
import pandas as pd
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import f1_score
import numpy as np
from sklearn import metrics
from pymystem3 import Mystem
m = Mystem()
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')
comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
# Очистим тексты от символов 
def clean_data(row):
    row = re.sub(r"(?:\n|\r)", " ", row)
    row = re.sub(r"[^a-zA-Z ]+", "", row).strip()
    row = row.lower()
    return row


comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
# Объявим корпус текстов, переведем его в тип юникод:
corpus = comments['text'].values #.astype('U')
print(corpus[0])

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


In [6]:
# Подключаем библиотеки лематизации анг текста
import nltk
nltk.download('wordnet')
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()


from tqdm import notebook
from tqdm import tqdm
import pymorphy2
from nltk.corpus import stopwords as nltk_stopwords

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               
                "N": wordnet.NOUN,              
                "V": wordnet.VERB,              
                "R": wordnet.ADV                
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [8]:
# Чистим текст и лематизируем
wnl = WordNetLemmatizer()

def clear_text(text):
    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = pattern.split()
    lemm = []
    for i in range(len(clear)):
        lemm.append(wnl.lemmatize(clear[i]))
    return " ".join(lemm)

In [9]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
for i in tqdm(range(len(corpus))):
    corpus[i] = lemm_text(corpus[i])

100%|██████████| 159292/159292 [24:47<00:00, 107.10it/s]


In [11]:
# Склеиваем лематизированный текст и наш датафрейм
df_corpus = pd.DataFrame(corpus)
comments['text'] = df_corpus
display(comments.head(10))


,Unnamed: 0,text,toxic
0,0,Explanation Why the edits make under my userna...,0
1,1,D'aww ! He match this background colour I 'm s...,0
2,2,"Hey man , I 'm really not try to edit war . It...",0
3,3,`` More I ca n't make any real suggestion on i...,0
4,4,"You , sir , be my hero . Any chance you rememb...",0
5,5,"`` Congratulations from me a well , use the to...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense ' be offensive to ...,0
9,9,alignment on this subject and which be contrar...,0


In [12]:

# Разделим датасет
df_train_valid, df_test = train_test_split(comments, test_size = 0.1, random_state = 12345)
df_train, df_valid = train_test_split(df_train_valid, shuffle=False, test_size=0.25, random_state = 12345)
print(df_train.shape, df_valid.shape, df_test.shape)



(107521, 3) (35841, 3) (15930, 3)


In [13]:
features_train = df_train['text']
target_train = df_train['toxic'].values

features_valid = df_valid['text']
target_valid = df_valid['toxic'].values

features_test = df_test['text']
target_test = df_test['toxic'].values

In [14]:
from nltk.corpus import stopwords 

try:
    nltk.download('stopwords')
except:
    pass
#Объявляю набор стоп-слов 
try:
    stopwords = set(stopwords.words('english'))
except:
    pass
#Объявляю TFIDF-векторизатор
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
#Выполняю векторизацию текстов
features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)
features_valid = count_tf_idf.transform(features_valid)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Вывод: После ознакомления с информацией я изабвился от лишних символов и стоп слов. Разделил датасет на выборки, лематизировал текст и изменил тип столбцов. Можно приступать к обучению модели.

## Обучение

In [15]:
# Пробуем модель логистической регрессии

lr = LogisticRegression(random_state=1, solver='liblinear', max_iter=100)
params = {
   'penalty':['l1', 'l2'],        
   'C':list(range(1,15,3)) 
}



lr_gs = GridSearchCV(lr, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)

print ("Best Params", lr_gs.best_params_)
print ("Best Score", lr_gs.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Params {'C': 4, 'penalty': 'l1'}
Best Score 0.7640425877676781


In [16]:
lr_best = LogisticRegression(random_state=1, class_weight = 'balanced', C = 4, penalty = 'l1', solver='liblinear', max_iter=100)
lr_best.fit(features_train, target_train)

LogisticRegression(C=4, class_weight='balanced', penalty='l1', random_state=1,
                   solver='liblinear')

In [17]:
pred1 = lr_best.predict(features_valid)
f1_score(target_valid, pred1)

0.7611752887995983

In [18]:
# Проверим теперь решающее дерево

tree = DecisionTreeClassifier(random_state = 123)
params = {
   'criterion':['gini', 'entropy'],        
   'max_depth':list(range(1,15,5)) 
}



tree_gs = GridSearchCV(tree, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)

print ("Best Params", tree_gs.best_params_)
print ("Best Score", tree_gs.best_score_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best Params {'criterion': 'gini', 'max_depth': 11}
Best Score 0.5974026524927939


In [19]:
tree_best = DecisionTreeClassifier(random_state = 123, criterion='gini', max_depth=11)
tree_best.fit(features_train, target_train)
pred2 = tree_best.predict(features_valid)
f1_score(target_valid, pred2)

0.6009675685361046

Лучшей моделью оказалась Логисчтическая регрессия можно приступать к финальному тесту.

## Выводы

In [20]:
pred1 = lr_best.predict(features_test)      
f1_lr = f1_score(target_test, pred1)     
f1_lr

0.7593397046046915

Вывод: После ознакомления с информацией я привел весь текст к нижнему регистру изабвился от лишних символов и стоп слов. Разделил датасет на выборки и изменил тип столбцов и приступил к обучению моделей. Я использовал две модели логисчической регрессии и решающего дерева. Подобрав лучшие параметры я приступил к тесту. У модели логистической регрессии оказались результаты лучше и провел финальный тест на тестовой выборке. Лучшая модель - Логисчтическая регрессия. Результат F1 мерки - 0.75


Вывод 2: Во время исправлений замечаний был выполнен большой объем работы и сменена последовательности действий. Сначало я лематизировал и очистил все твиты, после чего разделил на выборки, потом векторизировал выборки и приступил к обучению модели